In [2]:
import torch
from torch import nn
import torch.nn.functional as F

## basics

$$
\begin{split}
U_t&=R_t+\gamma R_{t+1}+\gamma^2 R_{t+2} + \cdots + \gamma^{n-t} R_n\\
&=\sum_{k=t}^n\gamma^{k-t}R_k{}\\
Q_\pi(s_t,a_t)&=\mathbb E\left[U_t|S_t=s_t,A_t=a_t\right]\\
&=\mathbb E_{S_{t+1},A_{t+1},\cdots, S_n,A_n}\left[U_t|S_t=s_t,A_t=a_t\right]\\
Q_\star(s_t,a_t)&=\max_\pi Q_\pi(s_t,a_t)
\end{split}
$$

- $U_t$: discounted return,
    - return: cumulative future reward
- $Q_\pi(s_t,a_t)$：action value function，动作价值函数
    - 其计算式中的期望消除了 $t$ 时刻之后的所有状态 $S_{t+1},\cdots, S_n$ 与所有动作 $A_{t+1}, \cdots, A_n$ 的随机性（也是 $U_t$ 随机性的来源）
        - 未来时刻的 $S_{t+1}, A_{t+1}$ 会带来 $R_{t+1}$
    - 对谁求期望就是消除对谁的随机性；
- $Q_\star(s_t,a_t)$：最优动作价值函数；

## DQN

$$
Q(s,a;w)
$$

- $w$ 表示 NN 的模型参数；
- DQN 的输入是 $s$，输出是离散动作空间 $\mathcal A$ 中的每个动作的 Q 值，是一个 scalar value；
- train DQN 的目标即是，对所有的 $s,a$ pair， DQN 对 $Q(s,a;w)$ 的预测尽量接近 $Q_\star(s,a)$
- 训练 DQN 最常用的算法是 TD（temporal difference，时间差分）

In [3]:
class QNet(nn.Module):
    def __init__(self, dim_state, num_action):
        super().__init__()
        self.fc1 = nn.Linear(dim_state, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_action)
    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## TD

- TD target 与 TD error

$$
\begin{split}
U_t&=R_t+\gamma R_{t+1}+\gamma^2 R_{t+2} + \cdots + \gamma^{n-t} R_n\\
&=\sum_{k=t}^n\gamma^{k-t}R_k\\
&=R_{t}+\sum_{k=t+1}^n\gamma^{k-t}R_{k}\\
&=R_{t}+\gamma\sum_{k=t+1}^n\gamma^{k-(t+1)}R_{k}\\
&=R_{t}+\gamma U_{t+1}\\
Q_\star(s_t,a_t)&=\max_\pi\mathbb E\left[U_t|S_t=s_t,A_t=a_t\right]\\
\end{split}
$$

- $U_t$ 的递归定义与计算
- 基于上述的两个等式可以推导出著名的最优贝尔曼方程（optimal Bellman equation）

### optimal Bellman equation

$$
Q_\star(s_t,a_t)=\mathbb E_{S_{t+1}\sim p(\cdot|s_t,a_t)}\left[R_t+\gamma\cdot\max_{\pi}Q_\star(S_{t+1}, A)\bigg|S_t=s_t,A_t=a_t\right]
$$

- 也是一种 recursive 的计算方式
- 右侧是一个期望，期望可以通过蒙特卡洛方法近似，当 Agent 在状态 $s_t$ 执行动作 $a_t$ 之后，环境通过状态转移函数 $p(s_{t+1}|s_t,a_t)$（mdp）计算出下一时刻的状态 $s_{t+1}$，因此当我们观测到 $s_t,a_t,s_{t+1}$ 时，奖励 $R_t$ 也会被观测到，记作 $r_t$，于是有了如下的四元组

$$
(s_t,a_t,r_t,s_{t+1})
$$

- 基于蒙特卡洛估计，可以算出等式右边期望的近似

$$
Q_\star(s_t,a_t) \approx r_t+\gamma\cdot \max_{a\in\mathcal A}Q_\star(s_{t+1},a)
$$

### 基于 TD train DQN

- 将最优动作价值函数 $Q_\star(s_t,a_t)$ 替换为神经网络 $Q(s,a;w)$

$$
\underbrace{Q(s_t,a_t;w)}_{预测 \hat{q}_t}\approx \underbrace{r_t + \gamma\cdot \max_{a\in\mathcal A}Q(s_{t+1},a;w)}_{\text{TD target}, \hat y_t}
$$

- 左边的 $Q(s_t,a_t;w)$ 是神经网络在 $t$ 时刻做出的预测：$\hat q_t$
- 右边的 TD target 则是神经网络在 $t+1$ 时刻做出的预测：$\hat y_t$，且基于了真实观测到的奖励 $r_t$（多了一部分事实）
- 对于右式 $\hat y_t$ 比着左式 $\hat q_t$ 多了部分事实，因此更可信，**在监督学习的范式下**，可以作为训练的 groud truth（将其视为常数），定义如下的损失


$$
\begin{split}
L(w)&=\frac12\left[Q(s_t,a_t;w)-\hat y_t\right]^2=\frac12\left[\hat q_t-\hat y_t\right]^2\\
\nabla_wL(w)&=\underbrace{(\hat q_t-\hat y_t)}_{\text{TD error}}\cdot \nabla_wQ(s_t,a_t;w)=\delta_t\cdot \nabla_wQ(s_t,a_t;w)
\end{split}
$$

- 梯度下降优化让 $\hat q_t$ 更接近 $\hat y_t$

$$
w\leftarrow w-\alpha\cdot\delta_t\nabla_wQ(s_t,a_t;w)
$$

### training process

- 给定一个四元组 $(s_t,a_t,r_t,s_{t+1})$ 一个 DQN 网络，我们可以得到

$$
\begin{split}
\hat q_t&=Q(s_t,a_t;w)\\
\hat y_t&=r_t+\max_{a\in\mathcal A}Q(s_{t+1},a;w)\\
\delta_t&=\hat q_t-\hat y_t
\end{split}
$$

- 梯度下降优化让 $\hat q_t$ 更接近 $\hat y_t$

$$
w\leftarrow w-\alpha\cdot\delta_t\nabla_wQ(s_t,a_t;w)
$$


- 训练 DQN 所需的四元组数据 $(s_t,a_t,r_t,s_{t+1})$ 与控制 agent 运动的策略无关，意味着可以用任何策略控制智能体与环境交互，同时记录下算法运动轨迹（trajectory），作为 DQN 的训练数据；
- 因此 DQN 的训练可以分为两个独立的部分
    - 收集训练数据
        - 可以用任意策略 $\pi$ 来控制智能体与环境交互，此时的 $\pi$ 可以称为行为策略，一种经典的行为策略是 $\epsilon$-greedy 策略
        
        $$
        a_t=\begin{cases}
        \arg\max_a Q(s_t,a;w), & r < 1-\epsilon (以概率 (1-\epsilon), )\\
        均匀采样 a\in \mathcal A, & r < \epsilon  (以概率 \epsilon)
        \end{cases}
        $$
        
        - 将 Agent 在一个 episode （回合） 中的轨迹记作
        
        $$
        (s_1,a_1,r_1), (s_2,a_2,r_2), (s_3,a_3,r_3), \cdots, (s_n,a_n,r_n)
        $$
        
        - 进一步将其划分为 $(s_t,a_t,r_t,s_{t+1})$ 这样的四元组，存入缓存，这个缓存就叫经验回放缓存（Experience Replay Buffer）
        
    - `forward/backward` 监督训练过程